# Proyecto de contrucción de pozos petroleros

La compañía de extracción de petróleo OilyGiant quiere encontrar los mejores lugares donde abrir 200 pozos nuevos de petróleo.

Para esta tarea, se realizarán los siguientes pasos:

- Leer los archivos con los parámetros recogidos de pozos petrolíferos en la región seleccionada: calidad de crudo y volumen de reservas.
- Crear un modelo para predecir el volumen de reservas en pozos nuevos.
- Elegir los pozos petrolíferos que tienen los valores estimados más altos.
- Elegir la región con el beneficio total más alto para los pozos petrolíferos seleccionados.

Se cuento con datos sobre muestras de crudo de tres regiones. Ya se conocen los parámetros de cada pozo petrolero de la región. Se creará un modelo que ayude a elegir la región con el mayor margen de beneficio. Analizaremos los beneficios y riesgos potenciales utilizando la técnica bootstrapping.

## Carga de datos y librerías

In [209]:
# Cargar todas las librerías
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split #Librería para poder dividir los dataframes
from sklearn.linear_model import LinearRegression #Libreria de regresión lineal
from sklearn.preprocessing import StandardScaler #Librería para hacer el escalado de características
#Librerías para calcular, la exactitud, valor de 1 y la curva precision-recall
from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle

### Descripción de los datos

**Características**

- id — identificador único de pozo de petróleo
- f0, f1, f2 — tres características de los puntos (su significado específico no es importante, pero las características en sí son significativas)

**Objetivo**

- product — volumen de reservas en el pozo de petróleo (miles de barriles). 

In [210]:
# Carga los archivos de datos 
try:
    df_0 = pd.read_csv('datasets/geo_data_0.csv')
    df_1 = pd.read_csv('datasets/geo_data_1.csv')
    df_2 = pd.read_csv('datasets/geo_data_2.csv')
except:
    df_0 = pd.read_csv('/datasets/geo_data_0.csv')
    df_1 = pd.read_csv('/datasets/geo_data_1.csv')
    df_2 = pd.read_csv('/datasets/geo_data_2.csv')

#### Geo Data 0

In [211]:
df_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [212]:
df_0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


#### Geo Data 1

In [213]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [214]:
df_1.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


#### Geo Data 2

In [215]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [216]:
df_2.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


Los tipos de datos para los 3 dataframes es adecuado.

## Análisis de datos

### Geo Data 0

In [217]:
na_ratio = ((df_0.isnull().sum() / len(df_0))*100).sort_values(ascending = False)
na_ratio

id         0.0
f0         0.0
f1         0.0
f2         0.0
product    0.0
dtype: float64

In [218]:
df_0.duplicated().sum()

0

### Geo Data 1

In [219]:
na_ratio = ((df_1.isnull().sum() / len(df_1))*100).sort_values(ascending = False)
na_ratio

id         0.0
f0         0.0
f1         0.0
f2         0.0
product    0.0
dtype: float64

In [220]:
df_1.duplicated().sum()

0

### Geo Data 2

In [221]:
na_ratio = ((df_2.isnull().sum() / len(df_2))*100).sort_values(ascending = False)
na_ratio

id         0.0
f0         0.0
f1         0.0
f2         0.0
product    0.0
dtype: float64

In [222]:
df_2.duplicated().sum()

0

Los dataframes no presentan datos ausentes ni datos duplicados.

## Entrenamiento y evaluación del modelo

### Segmentación de datos

Para las características usaremos las características f0, f1 y f2.

Para el objetivo usaremos la columna product.

In [223]:
features = df_0.drop(['id', 'product'], axis=1)
target = df_0['product']

Dividiremos el dataframe df_0 en 75% entrenamiento y 25% validación.

In [224]:
features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state=12345)

print('Porcentaje del conjunto de entrenamiento:',100*features_train.shape[0]/df_0.shape[0])
print('Porcentaje del conjunto de validación:',100*features_valid.shape[0]/df_0.shape[0])

Porcentaje del conjunto de entrenamiento: 75.0
Porcentaje del conjunto de validación: 25.0


### Escalado de características

Antes de entrenar el modelo procederemos a escalar los valores de f0, f1 y f2.

In [225]:
#Codigo utilizado para silenciar la alerta de python al escalar las características
pd.options.mode.chained_assignment = None

numeric = ['f0', 'f1', 'f2']

scaler = StandardScaler()
scaler.fit(features_train[numeric])
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_train.head()

,f0,f1,f2
27212,-0.544828,1.390264,-0.094959
7866,1.455912,-0.480422,1.209567
62041,0.260460,0.825069,-0.204865
70185,-1.837105,0.010321,-0.147634
82230,-1.299243,0.987558,1.273181


### Entrenamiento del modelo

Para este proyecto usaremos el algoritmo de machine learning conocido como regresión lineal.

In [226]:
model = LinearRegression() # inicializa el constructor de modelos
model.fit(features_train, target_train) # entrena el modelo en el conjunto de entrenamiento
predictions_valid = model.predict(features_valid) # obtén las predicciones del modelo en el conjunto de validación

### Evaluación del modelo

In [227]:
result = (mean_squared_error(target_valid, predictions_valid)) ** 0.5 # calcula la RECM en el conjunto de validación
print( "RECM del modelo de regresión lineal en el conjunto de validación:", result)

RECM del modelo de regresión lineal en el conjunto de validación: 37.5794217150813


In [228]:
mean = predictions_valid.mean()
print('El promedio de valores es:',mean)

El promedio de valores es: 92.59256778438035


### Creación de funciones y entrenamiento de los 3 datasets

In [229]:
def training(df, test_size=0.25, random_state=12345):
    features = df.drop(['id', 'product'], axis=1)
    target = df['product']
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=test_size, random_state=random_state)

    # Código utilizado para silenciar la alerta de python al escalar las características
    pd.options.mode.chained_assignment = None

    numeric = ['f0', 'f1', 'f2']
    scaler = StandardScaler()
    scaler.fit(features_train[numeric])
    features_train[numeric] = scaler.transform(features_train[numeric])
    features_valid[numeric] = scaler.transform(features_valid[numeric])

    model = LinearRegression() # inicializa el constructor de modelos
    model.fit(features_train, target_train) # entrena el modelo en el conjunto de entrenamiento
    predictions_valid = model.predict(features_valid) # obtén las predicciones del modelo en el conjunto de validación

    result = (mean_squared_error(target_valid, predictions_valid)) ** 0.5 # calcula la RECM en el conjunto de validación
    mean = predictions_valid.mean()
    target_valid = target_valid.reset_index(drop=True)
    
    predictions_valid = pd.Series(predictions_valid)

    return target_valid, result, mean, predictions_valid

**Dada la inversión de 100 millones por 200 pozos petrolíferos, de media un pozo petrolífero debe producir al menos un valor de 500,000 dólares en unidades para evitar pérdidas (esto es equivalente a 111.1 unidades).**

In [230]:
target_valid_0, result_0, mean_0, predictions_valid_0 = training(df_0)

# Imprime los resultados
print('El resultado de la RECM en el conjunto de validación es:', result_0)
print('El promedio de valores es:', mean_0)

El resultado de la RECM en el conjunto de validación es: 37.5794217150813
El promedio de valores es: 92.59256778438035


Para d_0 el valor promedio de volumen de reservas en los pozos petroleros es inferior al esperado para evitar pérdidas.

In [231]:
target_valid_1, result_1, mean_1, predictions_valid_1 = training(df_1)

# Imprime los resultados
print('El resultado de la RECM en el conjunto de validación es:', result_1)
print('El promedio de valores es:', mean_1)

El resultado de la RECM en el conjunto de validación es: 0.893099286775617
El promedio de valores es: 68.728546895446


Para d_1 el valor promedio de volumen de reservas en los pozos petroleros es inferior al esperado para evitar pérdidas e incluso inferior al valor promedio de d_0, por este motivo esta región no sería viable para el proyecto.

In [232]:
target_valid_2, result_2, mean_2, predictions_valid_2 = training(df_2)

# Imprime los resultados
print('El resultado de la RECM en el conjunto de validación es:', result_2)
print('El promedio de valores es:', mean_2)

El resultado de la RECM en el conjunto de validación es: 40.02970873393434
El promedio de valores es: 94.96504596800489


Para d_2 el valor promedio de volumen de reservas en los pozos petroleros es inferior al esperado para evitar pérdidas, pero es el valor más alto de las 3 regiones, por lo tanto, en este punto del proyecto se recomendaría el desarrollo de los pozos petroleros en esta región.

Para dar una mejor recomendación se seleccionarán los mejores 200 pozos petroleros para cada región y se calcularan las ganancias y beneficios.

## Cálculo de ganancias

In [233]:
def calc_income(predictions_valid, target_valid):
    predictions_valid = predictions_valid.reset_index(drop= True)
    target_valid = target_valid.reset_index(drop= True)
    best = predictions_valid.sort_values(ascending=False).head(200) # Nos quedamos con los 200 mejores pozos
    best = target_valid.iloc[best.index]
    best = pd.DataFrame(best, columns=['product'])
    
    best['income'] = best['product'].multiply(4500)
    total_income = best['income'].sum()
    
    return best, total_income    

In [234]:
best_0, total_income_0 = calc_income(predictions_valid_0, target_valid_0)
print('Las ganancias para los 200 mejores pozos de la región 0 sería de: ',total_income_0 - 100000000)

Las ganancias para los 200 mejores pozos de la región 0 sería de:  33208260.43139851


In [235]:
best_1, total_income_1 = calc_income(predictions_valid_1, target_valid_1)
print('El total de ingresos para la región 1 para los 200 mejores pozos sería de: ',total_income_1 - 100000000)

El total de ingresos para la región 1 para los 200 mejores pozos sería de:  24150866.966815084


In [236]:
best_2, total_income_2 = calc_income(predictions_valid_2, target_valid_2)
print('El total de ingresos para la región 2 para los 200 mejores pozos sería de: ',total_income_2 - 100000000)

El total de ingresos para la región 2 para los 200 mejores pozos sería de:  27103499.63599831


De las 3 regiones, la región 2 sería la que presentaría mayores ganancias potenciales, por un total estimado de 69.42 millones de dólares, por tanto, la recomendación sería contruir los pozos en esta región.

## Cálculo de riesgos

Usaremos la técnica bootstrapping con 1,000 muestras con un intervalo de confianza del 95% para calcular la distribución de los beneficios y obtener el beneficio promedio de cada región.

In [237]:
target_valid_0

0         10.038645
1        114.551489
2        132.603635
3        169.072125
4        122.325180
            ...    
24995    170.116726
24996     93.632175
24997    127.352259
24998     99.782700
24999    177.821022
Name: product, Length: 25000, dtype: float64

In [238]:
def bootstrapping(target_valid, predictions_valid):
    
    values = []
    for i in range(1000):
        
        target_subsample = target_valid.sample(n=500, random_state=i, replace=True)
        probs_subsample = predictions_valid.iloc[target_subsample.index]
        
        _,total_income = calc_income(probs_subsample, target_subsample)
        revenue = total_income - 100000000
        values.append(revenue)

    values = pd.Series(values)

    mean = values.mean()
    # Calcular el intervalo de confianza del 95%
    intervalo_confianza = np.percentile(values, [2.5, 97.5])
    
    probability_of_lose = np.mean(values < 0)
    risk = probability_of_lose * 100
    
    print(f'Beneficio promedio: ${mean:.2f}')
    print(f'Intervalo de confianza del 95%: (${intervalo_confianza[0]:.2f}, ${intervalo_confianza[1]:.2f})')
    print(f'Riesgo de pérdidas: {risk:.2f}%')

In [239]:
bootstrapping(target_valid_0, predictions_valid_0)

Beneficio promedio: $3883450.43
Intervalo de confianza del 95%: ($-998680.30, $8773652.89)
Riesgo de pérdidas: 6.60%


In [240]:
bootstrapping(target_valid_1, predictions_valid_1)

Beneficio promedio: $4573215.71
Intervalo de confianza del 95%: ($502583.70, $8723271.30)
Riesgo de pérdidas: 1.70%


In [241]:
bootstrapping(target_valid_2, predictions_valid_2)

Beneficio promedio: $3875514.95
Intervalo de confianza del 95%: ($-955100.56, $8813186.76)
Riesgo de pérdidas: 6.50%


## Conclusiones

Después de calcular el beneficio promedio para cada una de las regiones se encuentra que la región 1 sería la que generaría un beneficio promedio mayor con el menor porcentaje de riesgo, y por tanto, se toma como la región sugerida para el desarrollo de 200 pozos petroleros.

Como conclusión final, vemos que después de calcular el intervalo de confianza, el beneficio promedio y el porcentaje de riesgo usando bootstrapping cambiamos la región que resulta ser más apta para el desarrollo de 200 pozos petroleros.